## Datos Para entrenar red neuronal


In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#  Cargar los datos del archivo CSV
df = pd.read_csv('datos_variables.csv', sep=',')
# print(df.head())

# convertir la columna beta en una lista
beta = df['beta'].values.tolist()
print(beta)

# convertir la columna error_clearsky_solacast en una lista
error_clearsky_solacast = df['error_clearsky_solacast'].values.tolist()
print(error_clearsky_solacast)



[0.0, 0.2, 0.4, 0.2, 0.4, 0.5, 0.6, 0.3, 0.4, 0.1, 0.4, 0.4, 0.4, 0.1, 0.2, 0.7, 0.6, 0.3, 1.0, 0.1, 0.6]
[143.35525498438406, 74.36646894173451, 130.3588899702267, 74.15055258376864, 161.60389001848938, 183.767501158681, 97.28539823356904, 113.92722094201304, 413.79870651551033, 91.557596688379, 130.3588899702267, 109.64088173789912, 101.82379801716046, 219.87243806728603, 244.99774633402205, 115.76420660427966, 51.42473229831966, 113.92722094201304, 24.232146733698272, 155.12205803252058, 123.5717371930328]


In [33]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

def plot_signal(amplitude, frequency):
    t = np.linspace(0, 1, 1000)
    signal = amplitude * np.sin(2 * np.pi * frequency * t)
    
    plt.figure(figsize=(8, 3))
    plt.plot(t, signal)
    plt.title(f'Señal sinusoidal: Amplitud={amplitude}, Frecuencia={frequency}')
    plt.xlabel('Tiempo')
    plt.ylabel('Amplitud')
    plt.grid(True)
    plt.show()

# Definir sliders
amplitude_slider = widgets.FloatSlider(value=1, min=0, max=5, step=0.1, description='Amplitud:')
frequency_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='Frecuencia:')

# Crear la interfaz interactiva
interact(plot_signal, amplitude=amplitude_slider, frequency=frequency_slider)


interactive(children=(FloatSlider(value=1.0, description='Amplitud:', max=5.0), FloatSlider(value=1.0, descrip…

<function __main__.plot_signal(amplitude, frequency)>

## Análisis del modelo matematico propuesto

In [34]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
#  Cargar los datos del archivo CSV
df_SFV = pd.read_csv('../Data/Cleaned/SFV_2023.csv', sep=',')
df_solcast = pd.read_csv('../Data/Cleaned/solcast_2023.csv', sep=',')
Fecha = '2023-01-07'
row_day_solcast = df_solcast.loc[df_solcast['Date'] == Fecha]
row_day_fronius = df_SFV.loc[df_SFV['Date'] == Fecha]
# reiniciar el index
row_day_solcast = row_day_solcast.reset_index(drop=True)
row_day_fronius = row_day_fronius.reset_index(drop=True)
# Odenernar row_day_solcast de menor a mayor hora
row_day_solcast = row_day_solcast.sort_values(by='Time')
row_day_solcast = row_day_solcast.reset_index(drop=True)
# Crear un nuevo dataframe con los datos de solcast y fronius a partir del indice 
df = pd.DataFrame()
df['Time'] = row_day_solcast['Time']
df['GHI'] = row_day_solcast['GHI']
df['air_temp'] = row_day_solcast['air_temp']
df['cloud_opacity'] = row_day_solcast['cloud_opacity']
df['Producción_fotovoltaica_SFV'] = row_day_fronius['Producción_fotovoltaica_SFV']

In [48]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

def plot_signal(alpha, beta):
    #Parametros SFV
    efficiency = 0.2094  # Eficiencia del panel solar
    catidad_pv= 30
    area =2.274*1.134
    
    # normalizar 'cloud_opacity' de 0 a 1  en donde 1 es el valor más alto
    df['cloud_opacity'] = df['cloud_opacity'] / df['cloud_opacity'].max()
    # normalizar 'air_temp' de 0 a 1  en donde 1 es el valor más alto
    df['air_temp'] = df['air_temp'] / df['air_temp'].max()
    
    
    # Energía calculada a partir de Solcast 
    df['P_cloud_opacity_temp'] = df['GHI'] * efficiency *  area * catidad_pv * (1 - beta*df['cloud_opacity']) * (1 - alpha*df['air_temp'])

    # Energía calculada a partir de Fronius
    df['Potencia'] = df['Producción_fotovoltaica_SFV'] / (5/60)
    
    plt.figure(figsize=(15, 4))
    plt.plot(df['Time'], df['P_cloud_opacity_temp'], label='Solcast')
    plt.plot(df['Time'], df['Potencia'], label='Fronius')
    # grafico area bajo la curva
    plt.fill_between(df['Time'], df['P_cloud_opacity_temp'], df['Potencia'], color='gray', alpha=0.2)
    plt.title(f'Producción fotovoltaica calculada con: Alpha={alpha}, Beta={beta}')
    plt.xlabel('Tiempo')
    plt.ylabel('Amplitud')
    plt.legend()
    plt.xticks([100, 150, 200])
    # plt.grid(True)
    plt.show()
    # Energía calculada a partir de Fronius
    energia_fronius = df['Producción_fotovoltaica_SFV'].sum()
    # Horas de brillo solar
    hbs = energia_fronius / (30*0.54)
    print("Horas de brillo solar =", hbs/1000)
    print("Producción fotovoltaica SVF [kWh/día] =",energia_fronius/1000)
    # suma de la potencia generada solcast
    energia = df['P_cloud_opacity_temp']*(5/60)
    print("Energía calculada solacast [kWh] =",energia.sum()/1000)
    # error porcentual
    error = abs(  energia.sum() - energia_fronius.sum()) / energia_fronius.sum() * 100
    print('Error porcentual:', error, '%')

# Definir sliders
amplitude_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='alpha:')
frequency_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='beta:')

# Crear la interfaz interactiva
interact(plot_signal, alpha=amplitude_slider, beta=frequency_slider)

interactive(children=(FloatSlider(value=0.5, description='alpha:', max=1.0), FloatSlider(value=0.5, descriptio…

<function __main__.plot_signal(alpha, beta)>

In [49]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, DatePicker

def plot_signal(amplitude, frequency, selected_date):
    t = np.linspace(0, 1, 1000)
    signal = amplitude * np.sin(2 * np.pi * frequency * t)
    
    plt.figure(figsize=(8, 6))
    plt.plot(t, signal)
    plt.title(f'Señal sinusoidal: Amplitud={amplitude}, Frecuencia={frequency}, Fecha seleccionada: {selected_date}')
    plt.xlabel('Tiempo')
    plt.ylabel('Amplitud')
    plt.grid(True)
    plt.show()

# Definir sliders
amplitude_slider = widgets.FloatSlider(value=1, min=0, max=5, step=0.1, description='Amplitud:')
frequency_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='Frecuencia:')
date_picker = DatePicker(description='Fecha:')

# Crear la interfaz interactiva
interact(plot_signal, amplitude=amplitude_slider, frequency=frequency_slider, selected_date=date_picker)


interactive(children=(FloatSlider(value=1.0, description='Amplitud:', max=5.0), FloatSlider(value=1.0, descrip…

<function __main__.plot_signal(amplitude, frequency, selected_date)>